### -------------Yanolja crawling---------------

In [1]:
import scrapy, requests
from scrapy.http import TextResponse
import pandas as pd
import json

In [3]:
# 1. 프로젝트 생성

In [44]:
!rm -rf yanolja_hotel
!scrapy startproject yanolja_hotel

New Scrapy project 'yanolja_hotel', using template directory '/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/templates/project', created in:
    /home/ubuntu/python3/notebook/project/crawling/crawling-repo-2/in_progress/yanolja_hotel

You can start your first spider with:
    cd yanolja_hotel
    scrapy genspider example example.com


In [6]:
# 2. xpath 찾기 : headers 설정

In [ ]:
#호텔 key값 뽑아내기

In [2]:
import requests #호텔 key값 얻는 모듈 생성 -> yanolja.py
import datetime

def get_hotel_keys():    
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36", 
           "X-Requested-With": "XMLHttpRequest"
              }
    today = datetime.date.today()
    checkin_date = today.strftime("%Y-%m-%d")
    checkout_date = (today + datetime.timedelta(days=1)).strftime("%Y-%m-%d")
    region = "서울"
    num=0
    page = 1
    key_info = []
    while True:
        parent_url = f"https://www.yanolja.com/api/v1/contents/search?advert=KEYWORD&capacityAdult=2&capacityChild=0\
    &checkinDate={checkin_date}&checkoutDate={checkout_date}&hotel=1&keyword=서울&lat=37.50681&lng=127.06624&page={page}&limit=20&\
    searchKeyword={region}&searchType=hotel&sort=100&pathDivision=keyword-hotel&themes="
        req = requests.get(parent_url, headers=headers)
        response = req.json()
        #print('page=', str(apage))
        page += 1

        if response["motels"]["counts"] > 0:
            key = response["motels"]
            key_info.append(key)
        else:
            break
    hotel_keys = []
    for i in key_info:
        for j in i['lists']:
            hotel_keys.append(j['key'])        
    return hotel_keys

In [3]:
hotel_keys = get_hotel_keys()
len(hotel_keys)

416

### ------------xpath 확인 -> df확인 --------------

In [4]:
links = []
url = "https://www.yanolja.com/hotel/"
for hotel_key in hotel_keys:
    hotel_url = url + hotel_key
    links.append(hotel_url)
len(links)

416

In [5]:
# link 확인
link = links[0]
headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"}
req = requests.get(link, headers=headers)
response = TextResponse(req.url, body=req.text, encoding="utf-8")
response

<200 https://www.yanolja.com/hotel/3013410>

In [6]:
# xpath 확인
import json
location = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
location = json.loads(location)["props"]["pageProps"]["placeInfo"]["address"]
name = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/h1/text()')[0].extract()
level = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/div[1]/div[1]/text()')[0].extract()
score = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/div/span[1]/text()')[0].extract()
review_count = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/span/b/text()')[0].extract()
if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract() == []:
    room_type = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
else:    
    room_type = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract()
if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[2]/text()').extract() == []:
    room_detail = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
else:
    room_detail = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[2]/text()').extract()
price = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[4]/div/div[5]/span/text()').extract()
prices=[]
for num in range(len(room_type)):
    price = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
    price = json.loads(price)["props"]["pageProps"]["placeInfo"]["rooms"][num]["prices"][0]["price"]["discount"]["price"]
    prices.append(price)
        
location, name, level, score, review_count, room_type, room_detail, prices

('서울특별시 영등포구 경인로 870',
 '페어필드 바이 메리어트 서울',
 '4성급',
 '4.7',
 '8,823',
 ['Superior Single',
  'Premier Twin',
  'Premier King',
  'Premier Twin',
  'Premier Twin',
  'Premier King',
  'Premier Twin',
  'Premier Twin',
  'Premier King',
  'Premier Twin',
  'Premier King',
  'Premier King',
  'Premier King',
  'Superior Single'],
 ['룸온리',
  '특가야놀자☆After 21pm Check-in/연박불가',
  '특가야놀자☆After 21pm Check-in/연박불가',
  '특가야놀자☆선착순 고층 룸UP+27시간스테이',
  '특가야놀자☆선착순 고층 룸UP',
  '특가야놀자☆선착순 고층 룸UP+25시간',
  '특가야놀자☆선착순 고층 룸UP+25시간',
  '특가야놀자☆27시간스테이',
  '특가야놀자☆선착순 고층 룸UP',
  'stay for Breakfast',
  'stay for Breakfast',
  '특가야놀자☆선착순 고층 룸UP+27시간',
  '특가야놀자☆27시간스테이',
  'stay for Breakfast'],
 ['68,000원',
  '69,400원',
  '69,400원',
  '73,000원',
  '73,000원',
  '73,000원',
  '73,000원',
  '73,000원',
  '73,000원',
  '87,300원',
  '87,300원',
  '0원',
  '0원',
  '0원'])

str

In [147]:
# df 확인
locations, names, levels, scores, review_counts, room_types, room_details, prices_1, loading_urls = [], [], [], [], [] ,[], [], [], []

num1 = 0
for link in links[:10]:
    try:
        headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"}
        req = requests.get(link, headers=headers)
        response = TextResponse(req.url, body=req.text, encoding="utf-8")

        location = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
        location = json.loads(location)["props"]["pageProps"]["placeInfo"]["address"]
        name = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/h1/text()')[0].extract()
        level = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/div[1]/div[1]/text()')[0].extract()
        score = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/div/span[1]/text()')[0].extract()
        review_count = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/span/b/text()')[0].extract()
        if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract() == []:
            room_type = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
        else:    
            room_type = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract()
        if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[2]/text()').extract() == []:
            room_detail = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
        else:
            room_detail = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[2]/text()').extract()
        try:
            price = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[4]/div/div[5]/span/text()').extract()
            prices=[]
            for num in range(len(room_type)):
                price = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
                price = json.loads(price)["props"]["pageProps"]["placeInfo"]["rooms"][num]["prices"][0]["price"]["discount"]["price"]
                prices.append(price)
        except:
            prices= ["예약마감"]
    except:
        location = "페이지 없음"
        location = "페이지 없음"
        name = "페이지 없음"
        level = "페이지 없음"
        score = "페이지 없음"
        review_count = "페이지 없음"
        room_type = ["페이지 없음"]
        room_detail = ["페이지 없음"]
        price = ["페이지 없음"]
        prices= ["페이지 없음"]
        
    location = [location] * len(room_type)
    name = [name] * len(room_type)
    level = [level] * len(room_type)
    score = [score] * len(room_type)
    review_count = [review_count] * len(room_type)
    link = [link] * len(room_type)
    
    locations.extend(location)
    names.extend(name)
    levels.extend(level)
    scores.extend(score)
    review_counts.extend(review_count)
    room_types.extend(room_type)
    room_details.extend(room_detail)
    prices_1.extend(prices)
    loading_urls.extend(link)
    num1+=1
    print(str(num1) ,name[0])

1 페어필드 바이 메리어트 서울
2 나인트리 프리미어 명동2
3 영등포 GMS 호텔
4 신라스테이 구로
5 글래드 마포
6 노보텔 앰배서더 서울 용산
7 신라스테이 마포
8 더 파크호텔 서울
9 홀리데이인 익스프레스 서울 홍대
10 호텔 더 디자이너스 여의도


In [21]:
df = pd.DataFrame({
    'Name': names, 'Level': levels, 'Score':scores, 'Location': locations,
    'RoomType': room_types,'Roomdetail': room_details ,'Price': prices_1, 'Link': loading_urls, 'Review' : review_counts,
})
df.tail()

Name Level Score            Location        RoomType  \
203  더 파크호텔 서울   4성급   4.6  서울특별시 영등포구 양평로 136        파크 홈 스위트   
204  더 파크호텔 서울   4성급   4.6  서울특별시 영등포구 양평로 136    프리미어 패밀리 스위트   
205  더 파크호텔 서울   4성급   4.6  서울특별시 영등포구 양평로 136       파크 로얄 스위트   
206  더 파크호텔 서울   4성급   4.6  서울특별시 영등포구 양평로 136  패밀리 디럭스 트리플+욕조   
207  더 파크호텔 서울   4성급   4.6  서울특별시 영등포구 양평로 136       파크 듀얼 스위트   

                            Roomdetail     Price  \
203  마스크팩 무료 증정, 대중교통 이용시 커피 무료 이용권 제공  150,000원   
204  마스크팩 무료 증정, 대중교통 이용시 커피 무료 이용권 제공  150,000원   
205  마스크팩 무료 증정, 대중교통 이용시 커피 무료 이용권 제공  270,000원   
206  마스크팩 무료 증정, 대중교통 이용시 커피 무료 이용권 제공        0원   
207  마스크팩 무료 증정, 대중교통 이용시 커피 무료 이용권 제공        0원   

                                      Link Review  
203  https://www.yanolja.com/hotel/3016365  1,478  
204  https://www.yanolja.com/hotel/3016365  1,478  
205  https://www.yanolja.com/hotel/3016365  1,478  
206  https://www.yanolja.com/hotel/3016365  1,478  
207  https://www.yanolja.com/hotel/3016365  1,478

### --------확인 끝 spider.py로 확장---------

In [1]:
# 3. items.py

In [73]:
%%writefile yanolja_hotel/yanolja_hotel/items.py
import scrapy

class YanoljaHotelItem(scrapy.Item):
    location = scrapy.Field()
    name = scrapy.Field()
    level = scrapy.Field()
    score = scrapy.Field()
    review_count = scrapy.Field()
    link = scrapy.Field()
    room_type = scrapy.Field()
    price = scrapy.Field()
    platform = scrapy.Field()
    date = scrapy.Field()

Overwriting yanolja_hotel/yanolja_hotel/items.py


In [2]:
# 4. spider.py

In [3]:
%%writefile yanolja_hotel/yanolja_hotel/spiders/spider.py
import scrapy
from yanolja_hotel.items import YanoljaHotelItem
from yanolja import get_hotel_keys
import json
import datetime

class Spider(scrapy.Spider):
    name = "YanoljaHotel"
    allow_domain = ["https://www.yanolja.com"]
    start_urls = ["https://www.yanolja.com/hotel/"]
    
    def parse(self, response):
        hotel_keys = get_hotel_keys()
        start_urls = "https://www.yanolja.com/hotel/"
        links = []
        headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"}
        for hotel_key in hotel_keys:
            hotel_url = start_urls + hotel_key
            links.append(hotel_url)
        for link in links:
            yield scrapy.Request(link, headers=headers, callback=self.parse_content)
    
    def parse_content(self, response):
        item = YanoljaHotelItem()
        if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract() == []:
            room_types = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
        else:    
            room_types = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract()
        if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[2]/text()').extract() == []:
            room_detail = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
        else:
            room_detail = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[2]/text()').extract()
        
        room_type_all = list(zip(room_types, room_detail))
        roomtypes = []
        for idx in range(len(room_type_all)):
            room_type_all[idx] = ",".join(room_type_all[idx])
            roomtypes.append(room_type_all[idx])   

        prices=[]
        for num in range(len(roomtypes)):
            price = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
            price = json.loads(price)["props"]["pageProps"]["placeInfo"]["rooms"][num]["prices"][0]["price"]["discount"]["price"]
            price = int(price.strip("원").replace(",",""))
            prices.append(price)

        locations = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
        locations = json.loads(locations)["props"]["pageProps"]["placeInfo"]["address"]
        names = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/h1/text()')[0].extract()
        levels = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/div[1]/div[1]/text()')[0].extract()
        scores = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/div/span[1]/text()')[0].extract()
        review_counts = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/span/b/text()')[0].extract()
        review_counts = int(review_counts.replace(",",""))
        links = response.url
        platforms = "야놀자"
        today = datetime.date.today()
        today = str(today)
        
        for i in range(len(roomtypes)):
            
            item["platform"] = platforms
            item["location"] = locations
            item["name"] = names
            item["level"] = levels
            item["score"] = float(scores)
            item["review_count"] = review_counts
            item["link"] = links
            item["date"] = today
            item["room_type"] = roomtypes[i]
            item["price"] = prices[i]
            
            yield item

Overwriting yanolja_hotel/yanolja_hotel/spiders/spider.py


In [4]:
# 5. settings.py : robots.txt 여부 파악 

In [5]:
# 6. 프로젝트 실행

In [36]:
%pwd

'/home/ubuntu/python3/notebook/project/crawling/crawling-repo-2/in_progress'

In [1]:
%%writefile runyanolja.sh
cd ~/python3/notebook/project/crawling/crawling-repo-2/in_progress/yanolja_hotel
rm yanolja_hotel.csv
scrapy crawl YanoljaHotel -o yanolja_hotel.csv

Writing runyanolja.sh


In [ ]:
!/bin/bash run.sh

In [78]:
import pandas as pd
data = pd.read_csv('./yanolja_hotel/yanolja_hotel.csv')

In [79]:
data

date level                                   link  \
0     2021-03-11   3성급  https://www.yanolja.com/hotel/3011807   
1     2021-03-11   3성급  https://www.yanolja.com/hotel/3011807   
2     2021-03-11   3성급  https://www.yanolja.com/hotel/3011807   
3     2021-03-11   3성급  https://www.yanolja.com/hotel/3011807   
4     2021-03-11   3성급  https://www.yanolja.com/hotel/3011807   
5     2021-03-11   3성급  https://www.yanolja.com/hotel/3011807   
6     2021-03-11   3성급  https://www.yanolja.com/hotel/3011807   
7     2021-03-11   3성급  https://www.yanolja.com/hotel/3011807   
8     2021-03-11   3성급  https://www.yanolja.com/hotel/3011807   
9     2021-03-11   4성급  https://www.yanolja.com/hotel/3015395   
10    2021-03-11   4성급  https://www.yanolja.com/hotel/3001680   
11    2021-03-11   3성급  https://www.yanolja.com/hotel/3008549   
12    2021-03-11   3성급  https://www.yanolja.com/hotel/3011807   
13    2021-03-11   3성급  https://www.yanolja.com/hotel/3011807   
14    2021-03-11   3성급  https://www.yanolja.com/hotel/3011807   
15    2021-03-11   3성급  https://www.yanolja.com/hotel/3011807   
16    2021-03-11   3성급  https://www.yanolja.com/hotel/3011807   
17    2021-03-11   3성급  https://www.yanolja.com/hotel/3011807   
18    2021-03-11   3성급  https://www.yanolja.com/hotel/3011807   
19    2021-03-11   4성급  https://www.yanolja.com/hotel/3013410   
20    2021-03-11   4성급  https://www.yanolja.com/hotel/3015395   
21    2021-03-11   4성급  https://www.yanolja.com/hotel/3015395   
22    2021-03-11   4성급  https://www.yanolja.com/hotel/3015395   
23    2021-03-11   4성급  https://www.yanolja.com/hotel/3015395   
24    2021-03-11   4성급  https://www.yanolja.com/hotel/3015395   
25    2021-03-11   4성급  https://www.yanolja.com/hotel/3015395   
26    2021-03-11   4성급  https://www.yanolja.com/hotel/3015395   
27    2021-03-11   4성급  https://www.yanolja.com/hotel/3001680   
28    2021-03-11   4성급  https://www.yanolja.com/hotel/3001680   
29    2021-03-11   4성급  https://www.yanolja.com/hotel/3001680   
...          ...   ...                                    ...   
3386  2021-03-11   3성급  https://www.yanolja.com/hotel/3001564   
3387  2021-03-11   3성급  https://www.yanolja.com/hotel/3001564   
3388  2021-03-11   4성급  https://www.yanolja.com/hotel/3009497   
3389  2021-03-11   4성급  https://www.yanolja.com/hotel/3009497   
3390  2021-03-11   4성급  https://www.yanolja.com/hotel/3009497   
3391  2021-03-11   4성급  https://www.yanolja.com/hotel/3009497   
3392  2021-03-11   4성급  https://www.yanolja.com/hotel/3009497   
3393  2021-03-11   4성급  https://www.yanolja.com/hotel/3009497   
3394  2021-03-11   4성급  https://www.yanolja.com/hotel/3009497   
3395  2021-03-11   4성급  https://www.yanolja.com/hotel/3009497   
3396  2021-03-11   4성급  https://www.yanolja.com/hotel/3009497   
3397  2021-03-11   4성급  https://www.yanolja.com/hotel/3009497   
3398  2021-03-11   4성급  https://www.yanolja.com/hotel/3009497   
3399  2021-03-11   4성급  https://www.yanolja.com/hotel/3009497   
3400  2021-03-11   4성급  https://www.yanolja.com/hotel/3009497   
3401  2021-03-11   4성급  https://www.yanolja.com/hotel/3009497   
3402  2021-03-11   4성급  https://www.yanolja.com/hotel/3009497   
3403  2021-03-11   4성급  https://www.yanolja.com/hotel/3009497   
3404  2021-03-11   4성급  https://www.yanolja.com/hotel/3009497   
3405  2021-03-11   4성급  https://www.yanolja.com/hotel/3009497   
3406  2021-03-11   4성급  https://www.yanolja.com/hotel/3009497   
3407  2021-03-11   4성급  https://www.yanolja.com/hotel/3009497   
3408  2021-03-11   4성급  https://www.yanolja.com/hotel/3009497   
3409  2021-03-11   4성급  https://www.yanolja.com/hotel/3009497   
3410  2021-03-11   4성급  https://www.yanolja.com/hotel/3009497   
3411  2021-03-11   4성급  https://www.yanolja.com/hotel/3009497   
3412  2021-03-11   4성급  https://www.yanolja.com/hotel/3009497   
3413  2021-03-11   4성급  https://www.yanolja.com/hotel/3009497   
3414  2021-03-11   4성급  https://www.yanolja.com/hotel/3009497   
3415  2021-03-11   4성급  https://www.yanolja.com

In [90]:
# 7. pipeline.py : 크롤링한 데이터를 mongodb 저장

In [ ]:
# scrapy 프로젝트에 pymongo 모듈 생성

In [12]:
%%writefile yanolja_hotel/yanolja_hotel/mongodb.py
import pymongo

client = pymongo.MongoClient("mongodb://15.165.254.37:27017")
collection = client.hotel.hotel_info

Overwriting yanolja_hotel/yanolja_hotel/mongodb.py


In [ ]:
# pipeline.py 설정

In [13]:
%%writefile yanolja_hotel/yanolja_hotel/pipelines.py
from itemadapter import ItemAdapter
from .mongodb import collection

class YanoljaHotelPipeline:
    def process_item(self, item, spider):
        data = {"date": item["date"], "platform": item["platform"], "name": item["name"],
                "level": item["level"], "score": item["score"], "review": item["review_count"],
                "location": item["location"], "room_type": item["room_type"],
                "price": item["price"],"link": item["link"]}
        collection.insert(data)
        return item

Overwriting yanolja_hotel/yanolja_hotel/pipelines.py


In [89]:
# 8. settings.py -> pipeline 설정 변경

```
ITEM_PIPELINES = {
   'yanolja_hotel.pipelines.YanoljaHotelPipeline': 300,
}
```

In [ ]:
# 9. 프로젝트 실행!

In [4]:
!/bin/bash runyanolja.sh

2021-03-14 19:48:58 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: yanolja_hotel)
2021-03-14 19:48:58 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Jan 15 2021, 05:57:06) - [GCC 7.5.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1i  8 Dec 2020), cryptography 3.3.1, Platform Linux-5.4.0-1038-aws-x86_64-with-debian-buster-sid
2021-03-14 19:48:58 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-03-14 19:48:58 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'yanolja_hotel',
 'NEWSPIDER_MODULE': 'yanolja_hotel.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['yanolja_hotel.spiders']}
2021-03-14 19:48:59 [scrapy.extensions.telnet] INFO: Telnet Password: a091600a165e832b
2021-03-14 19:48:59 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusa

2021-03-14 19:49:02 [urllib3.connectionpool] DEBUG: https://www.yanolja.com:443 "GET /api/v1/contents/search?advert=KEYWORD&capacityAdult=2&capacityChild=0%20%20%20%20&checkinDate=2021-03-14&checkoutDate=2021-03-15&hotel=1&keyword=%EC%84%9C%EC%9A%B8&lat=37.50681&lng=127.06624&page=11&limit=20&%20%20%20%20searchKeyword=%EC%84%9C%EC%9A%B8&searchType=hotel&sort=100&pathDivision=keyword-hotel&themes= HTTP/1.1" 200 None
2021-03-14 19:49:02 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.yanolja.com:443
2021-03-14 19:49:02 [urllib3.connectionpool] DEBUG: https://www.yanolja.com:443 "GET /api/v1/contents/search?advert=KEYWORD&capacityAdult=2&capacityChild=0%20%20%20%20&checkinDate=2021-03-14&checkoutDate=2021-03-15&hotel=1&keyword=%EC%84%9C%EC%9A%B8&lat=37.50681&lng=127.06624&page=12&limit=20&%20%20%20%20searchKeyword=%EC%84%9C%EC%9A%B8&searchType=hotel&sort=100&pathDivision=keyword-hotel&themes= HTTP/1.1" 200 None
2021-03-14 19:49:02 [urllib3.connectionpool] DEBUG: Sta

2021-03-14 19:49:05 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001791>
{'date': '2021-03-14',
 'level': '비즈니스',
 'link': 'https://www.yanolja.com/hotel/3001791',
 'location': '서울특별시 중구 장충단로13길 20',
 'name': '호텔 스카이파크 킹스타운 동대문점',
 'platform': '야놀자',
 'price': 63300,
 'review_count': 1655,
 'room_type': '주차불가,스탠다드 더블,특가야놀자☆/9시 얼리체크인, 취소불가',
 'score': 4.7}
2021-03-14 19:49:05 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3000643>
{'date': '2021-03-14',
 'level': '4성급',
 'link': 'https://www.yanolja.com/hotel/3000643',
 'location': '서울특별시 중구 마른내로 71',
 'name': '호텔 피제이 명동',
 'platform': '야놀자',
 'price': 63300,
 'review_count': 1689,
 'room_type': '스탠다드 트윈,특가야놀자☆',
 'score': 4.7}
2021-03-14 19:49:05 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3011846>
{'date': '2021-03-14',
 'level': '4성급',
 'link': 'https://www.yanolja.com/hotel/3011846',
 'location': '서울특별시 강남구 테헤란로 415',
 'name': 'L7 강남 바이 롯

2021-03-14 19:49:05 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3013410>
{'date': '2021-03-14',
 'level': '4성급',
 'link': 'https://www.yanolja.com/hotel/3013410',
 'location': '서울특별시 영등포구 경인로 870',
 'name': '페어필드 바이 메리어트 서울',
 'platform': '야놀자',
 'price': 60850,
 'review_count': 8864,
 'room_type': 'Premier Twin,특가야놀자☆After 21pm Check-in/연박불가',
 'score': 4.7}
2021-03-14 19:49:05 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/1000102261>
{'date': '2021-03-14',
 'level': '4성급',
 'link': 'https://www.yanolja.com/hotel/1000102261',
 'location': '서울특별시 종로구 인사동길 49',
 'name': '나인트리 프리미어 호텔 인사동',
 'platform': '야놀자',
 'price': 66000,
 'review_count': 1475,
 'room_type': '스탠다드 트윈,야놀자단독ㅣ연박전용 Green Earth by Nine Tree',
 'score': 4.8}
2021-03-14 19:49:05 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001791>
{'date': '2021-03-14',
 'level': '비즈니스',
 'link': 'https://www.yanolja.com/hotel/3001791',
 'locat

2021-03-14 19:49:05 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/3001075> (referer: https://www.yanolja.com/hotel/)
2021-03-14 19:49:05 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/3001558> (referer: https://www.yanolja.com/hotel/)
2021-03-14 19:49:05 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/3007477> (referer: https://www.yanolja.com/hotel/)
2021-03-14 19:49:05 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3013410>
{'date': '2021-03-14',
 'level': '4성급',
 'link': 'https://www.yanolja.com/hotel/3013410',
 'location': '서울특별시 영등포구 경인로 870',
 'name': '페어필드 바이 메리어트 서울',
 'platform': '야놀자',
 'price': 60850,
 'review_count': 8864,
 'room_type': 'Premier King,특가야놀자☆After 21pm Check-in/연박불가',
 'score': 4.7}
2021-03-14 19:49:05 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3013410>
{'date': '2021-03-14',
 'level': '4성급',
 'link': 'https

...

2021-03-14 19:49:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3015391>
{'date': '2021-03-14',
 'level': '5성급',
 'link': 'https://www.yanolja.com/hotel/3015391',
 'location': '서울특별시 용산구 청파로20길 95',
 'name': '노보텔 앰배서더 서울 용산',
 'platform': '야놀자',
 'price': 187500,
 'review_count': 2831,
 'room_type': '슈페리어 더블,19금 딸기타워 PKG',
 'score': 4.7}
2021-03-14 19:49:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3015391>
{'date': '2021-03-14',
 'level': '5성급',
 'link': 'https://www.yanolja.com/hotel/3015391',
 'location': '서울특별시 용산구 청파로20길 95',
 'name': '노보텔 앰배서더 서울 용산',
 'platform': '야놀자',
 'price': 188100,
 'review_count': 2831,
 'room_type': '슈페리어 트윈,2인 조식 포함',
 'score': 4.7}
2021-03-14 19:49:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3015391>
{'date': '2021-03-14',
 'level': '5성급',
 'link': 'https://www.yanolja.com/hotel/3015391',
 'location': '서울특별시 용산구 청파로20길 95',
 'name': '노보텔 앰배서더 서울 용산',

2021-03-14 19:49:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001415>
{'date': '2021-03-14',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3001415',
 'location': '서울특별시 마포구 마포대로 83',
 'name': '신라스테이 마포',
 'platform': '야놀자',
 'price': 104500,
 'review_count': 3206,
 'room_type': '디럭스 더블,One More Stay 1+1 PKG, 17-26층',
 'score': 4.6}
2021-03-14 19:49:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3008569>
{'date': '2021-03-14',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3008569',
 'location': '서울특별시 금천구 서부샛길 584',
 'name': '골드리버 호텔',
 'platform': '야놀자',
 'price': 65200,
 'review_count': 2603,
 'room_type': '디럭스 더블,특가야놀자☆늦은체크아웃14시',
 'score': 4.8}
2021-03-14 19:49:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3008569>
{'date': '2021-03-14',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3008569',
 'location': '서울특별시 금천구 서부샛길 584',
 'name': '골드리버 호텔',
 '

2021-03-14 19:49:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3009497>
{'date': '2021-03-14',
 'level': '4성급',
 'link': 'https://www.yanolja.com/hotel/3009497',
 'location': '서울특별시 중구 마른내로 28',
 'name': '나인트리 프리미어 명동2',
 'platform': '야놀자',
 'price': 75000,
 'review_count': 2228,
 'room_type': '스탠다드 트윈,1.5박+30시간 STAYㅣ5객실 한정, 주차불가',
 'score': 4.8}
2021-03-14 19:49:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3009497>
{'date': '2021-03-14',
 'level': '4성급',
 'link': 'https://www.yanolja.com/hotel/3009497',
 'location': '서울특별시 중구 마른내로 28',
 'name': '나인트리 프리미어 명동2',
 'platform': '야놀자',
 'price': 75000,
 'review_count': 2228,
 'room_type': '스탠다드 트리플,주차불가',
 'score': 4.8}
2021-03-14 19:49:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3009497>
{'date': '2021-03-14',
 'level': '4성급',
 'link': 'https://www.yanolja.com/hotel/3009497',
 'location': '서울특별시 중구 마른내로 28',
 'name': '나인트리 프리미어 명동2',
 

2021-03-14 19:49:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/3000969> (referer: https://www.yanolja.com/hotel/)
2021-03-14 19:49:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001564>
{'date': '2021-03-14',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3001564',
 'location': '서울특별시 동작구 시흥대로 596',
 'name': '신라스테이 구로',
 'platform': '야놀자',
 'price': 61710,
 'review_count': 4141,
 'room_type': '스탠다드 → 패밀리트윈 무료 업그레이드,룸UP',
 'score': 4.6}
2021-03-14 19:49:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001564>
{'date': '2021-03-14',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3001564',
 'location': '서울특별시 동작구 시흥대로 596',
 'name': '신라스테이 구로',
 'platform': '야놀자',
 'price': 61710,
 'review_count': 4141,
 'room_type': '스탠다드 트윈 → 디럭스 트윈 무료 업그레이드,룸UP',
 'score': 4.6}
2021-03-14 19:49:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001564>
{'

2021-03-14 19:49:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001564>
{'date': '2021-03-14',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3001564',
 'location': '서울특별시 동작구 시흥대로 596',
 'name': '신라스테이 구로',
 'platform': '야놀자',
 'price': 102850,
 'review_count': 4141,
 'room_type': '스탠다드 트윈,서프라이즈 패키지-석식',
 'score': 4.6}
2021-03-14 19:49:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001564>
{'date': '2021-03-14',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3001564',
 'location': '서울특별시 동작구 시흥대로 596',
 'name': '신라스테이 구로',
 'platform': '야놀자',
 'price': 106480,
 'review_count': 4141,
 'room_type': '스탠다드 더블,서프라이즈 패키지-석식',
 'score': 4.6}
2021-03-14 19:49:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001564>
{'date': '2021-03-14',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3001564',
 'location': '서울특별시 동작구 시흥대로 596',
 'name': '신라스테이 구로',
 'platform': '야놀

In [91]:
# 10.crontab 실행